In [11]:
# Get the training, and split the file, and galactic change.
filename = "../../data-files/wikitext-15M-conllu/wikitext-15M-test.conllu"
with open(filename) as f:
    content = f.readlines()

In [12]:
count = 0
for c in content:
    if len(c.strip()) == 0:
        count += 1

In [14]:
content[:20]

['1\t=\t_\tPUNCT\tNFP\t_\t0\t_\t_\tstart_char=0|end_char=1\n',
 '2\tRobert\t_\tPROPN\tNNP\tNumber=Sing\t1\t_\t_\tstart_char=2|end_char=8\n',
 '3\tBoulter\t_\tPROPN\tNNP\tNumber=Sing\t2\t_\t_\tstart_char=9|end_char=16\n',
 '4\t=\t_\tPUNCT\t,\t_\t3\t_\t_\tstart_char=17|end_char=18\n',
 '\n',
 '1\tRobert\t_\tPROPN\tNNP\tNumber=Sing\t0\t_\t_\tstart_char=0|end_char=6\n',
 '2\tBoulter\t_\tPROPN\tNNP\tNumber=Sing\t1\t_\t_\tstart_char=7|end_char=14\n',
 '3\tis\t_\tAUX\tVBZ\tMood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin\t2\t_\t_\tstart_char=15|end_char=17\n',
 '4\tan\t_\tDET\tDT\tDefinite=Ind|PronType=Art\t3\t_\t_\tstart_char=18|end_char=20\n',
 '5\tEnglish\t_\tADJ\tJJ\tDegree=Pos\t4\t_\t_\tstart_char=21|end_char=28\n',
 '6\tfilm\t_\tNOUN\tNN\tNumber=Sing\t5\t_\t_\tstart_char=29|end_char=33\n',
 '7\t,\t_\tPUNCT\t,\t_\t6\t_\t_\tstart_char=34|end_char=35\n',
 '8\ttelevision\t_\tNOUN\tNN\tNumber=Sing\t7\t_\t_\tstart_char=36|end_char=46\n',
 '9\tand\t_\tCCONJ\tCC\t_\t8\t_\t_\tstart_char=47|

In [1]:
# We will run the following line to convert our conllu file using JAVA.
# types of shifts:
# wiki-text (galactic-en~fr@N~fr@V)
# wiki-text (galactic-en~ja_ktc@N~ja_ktc@V)
# wiki-text (galactic-en~fr@N~ja_ktc@V)

In [5]:
! GALACTIC_ROOT=../../submodules/gdtreebank/ \
../../submodules/gdtreebank/bin/gd-translate \
--input ../../data-files/wikitext-15M-conllu/wikitext-15M-train.conllu \
--spec en~fr@N~fr@V

# --input ../../data-files/wikitext-15M-conllu/wikitext-15M-train.conllu \
# --input ../../data-files/wikitext-15M-conllu/wikitext-15M-test.conllu \
# --input ../../data-files/wikitext-15M-conllu/wikitext-15M-validation.conllu \

java -cp ../../submodules/gdtreebank//bin/gdgen.jar -Xmx8g datagen.GalacticGen --task test --inputTB ../../data-files/wikitext-15M-conllu/wikitext-15M-train.conllu --outputTB ../../data-files/wikitext-15M-conllu/wikitext-15M-train-en~fr@N~fr@V.conllu --verbose 1 --seed 0 --supStrateModelNOUN ../../submodules/gdtreebank//models/GD_French/fr@N.orm --subStrateModelNOUN ../../submodules/gdtreebank//models/GD_English/en@N.orm --supStrateModelVERB ../../submodules/gdtreebank//models/GD_French/fr@V.orm --subStrateModelVERB ../../submodules/gdtreebank//models/GD_English/en@V.orm
0        INFO                GalacticGen - Running with args: --task test --inputTB ../../data-files/wikitext-15M-conllu/wikitext-15M-train.conllu --outputTB ../../data-files/wikitext-15M-conllu/wikitext-15M-train-en~fr@N~fr@V.conllu --verbose 1 --seed 0 --supStrateModelNOUN ../../submodules/gdtreebank//models/GD_French/fr@N.orm --subStrateModelNOUN ../../submodules/gdtreebank//models/GD_English/en@N.orm --supStrateMod